In [ ]:
!pip install sgp4
!pip install spacetrack
!pip install pandas
!pip install astropy

     |████████████████████████████████| 258 kB 12.8 MB/s 
     |████████████████████████████████| 85 kB 3.7 MB/s 
  Created wheel for logbook: filename=Logbook-1.5.3-cp37-cp37m-linux_x86_64.whl size=66396 sha256=a51c81731081dad6204a1a2f3853c97df8ef9f732fbdbee55e35e31c887a14cb
  Stored in directory: /root/.cache/pip/wheels/95/4b/a3/fc9b9d17a06a956767a8dc74058ba64d4306653df2289094cc
Successfully built logbook


In [ ]:
!cd nasa-space-apps-challenge-2021 && git checkout data && git push https://TOKEN@github.com/luamzi/nasa-space-apps-challenge-2021

Branch 'data' set up to track remote branch 'data' from 'origin'.
Switched to a new branch 'data'
Everything up-to-date


In [ ]:
!git config --global user.email "your@email"
!git config --global user.name "yourUsername"

In [ ]:
import threading
import http.server
import socketserver

PORT = 8000

handler = http.server.SimpleHTTPRequestHandler

def myserver():
  with socketserver.TCPServer(("", PORT), handler) as httpd:
      print("Server started at localhost:" + str(PORT))
      httpd.serve_forever()

main_thread = threading.Thread(target=myserver)
main_thread.start()

Server started at localhost:8000


In [ ]:
!git clone https://github.com/luamzi/nasa-space-apps-challenge-2021.git

fatal: destination path 'nasa-space-apps-challenge-2021' already exists and is not an empty directory.


In [ ]:
!cd nasa-space-apps-challenge-2021 && git checkout data

Already on 'data'
Your branch is up to date with 'origin/data'.


In [ ]:
import time

# Counting time elapsed
start_count = time.time()

import os
import json
import datetime as dt
from sgp4.api import jday
from sgp4.api import Satrec
from sgp4.api import SGP4_ERRORS
import pandas as pd

import spacetrack.operators as op
from spacetrack import SpaceTrackClient

from astropy.time import Time
from astropy.coordinates import TEME, CartesianDifferential, CartesianRepresentation
from astropy import units as u
from astropy.coordinates import ITRS

file_path = os.getcwd() + "/data"
file_path = "nasa-space-apps-challenge-2021/data.json"

st = SpaceTrackClient(identity='hitalo.c.a@gmail.com', password='EuMeChamoHitalo2000!')

while (True):
  debris_list = []
  data = st.tle_latest(iter_lines=True, ordinal=1, epoch='>now-30',
                      mean_motion=op.inclusive_range(0.99, 1.01),
                      eccentricity=op.less_than(0.01), format='tle')

  with open('tle_latest.txt', 'w') as fp:
      for line in data:
          fp.write(line + '\n')

  tle_latest = pd.read_csv("tle_latest.txt", header=None)

  s_array = tle_latest[tle_latest.index%2 == 0].values.tolist()
  t_array = tle_latest[tle_latest.index%2 != 0].values.tolist()

  e, r, v = [], [], []
  lat, lon, hgt = [], [], []
  print('Converting tle data...')

  #indexfile = open("/var/www/html/index2.html", "w")
  indexfile = open(file_path, "w")

  for i in range(len(s_array)):
    #print(i)
    timestamp_now = dt.datetime.now()
    jd, fr = jday(timestamp_now.year, timestamp_now.month, timestamp_now.day, timestamp_now.hour, timestamp_now.minute, timestamp_now.second)
    
    satellite = Satrec.twoline2rv(s_array[i][0], t_array[i][0])
    e_, r_, v_ = satellite.sgp4(jd, fr)
    if e_ != 0:
      print(f'Error: {SGP4_ERRORS[e_]}')
    else:
      e.append(e_)
      r.append(r_)
      v.append(v_)

      t = Time(jd, format='jd')
      #print(t)

      teme_p = r_
      teme_v = v_

      #from astropy.coordinates import TEME, CartesianDifferential, CartesianRepresentation
      #from astropy import units as u
      teme_p = CartesianRepresentation(teme_p*u.km)
      teme_v = CartesianDifferential(teme_v*u.km/u.s)
      teme = TEME(teme_p.with_differentials(teme_v), obstime=t)

      #from astropy.coordinates import ITRS
      itrs = teme.transform_to(ITRS(obstime=t))  
      location = itrs.earth_location
      #location.geodetic
      
      lon_ = location.geodetic.lon.deg
      lat_ = location.geodetic.lat.deg
      hgt_ = location.geodetic.height.value

      lon.append(lon_)
      lat.append(lat_)
      hgt.append(hgt_)

      
      debris_list.append({"id":i, "timestamp":str(timestamp_now), "lat":lat_, "lon":lon_, "height":hgt_})
      
      #indexfile.write(str(lon_) + "," + str(lat_) + "," + str(hgt_) + "\n")

  json_line = {"debris": debris_list}
  json.dump(json_line, indexfile, ensure_ascii=False, indent=4)
  indexfile.close()
  !cd nasa-space-apps-challenge-2021 && git checkout data && git add --all && git commit -m "Update data" && git push https://TOKEN@github.com/luamzi/nasa-space-apps-challenge-2021
  print('Done!')

  # End time counting
  print("Time elapsed:" + str(time.time() - start_count))

Streaming output truncated to the last 5000 lines.
[data c2f1be4] Update data
 1 file changed, 7676 insertions(+), 7676 deletions(-)
 rewrite data.json (76%)
Counting objects: 3, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 49.82 KiB | 3.56 MiB/s, done.
Total 3 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/luamzi/nasa-space-apps-challenge-2021
   803be36..c2f1be4  data -> data
Done!
Time elapsed:10464.19584774971
Converting tle data...
M	data.json
Already on 'data'
Your branch is ahead of 'origin/data' by 494 commits.
  (use "git push" to publish your local commits)
[data 9e368e5] Update data
 1 file changed, 7676 insertions(+), 7676 deletions(-)
 rewrite data.json (76%)
Counting objects: 3, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 49.52 KiB | 3.30 MiB/s, done.
Total 3 (